In [10]:
from math import floor
import numpy as np
import matplotlib.pyplot as plt
from ase.db import connect
from scipy.sparse import identity
from scipy.sparse.linalg import eigs
import pymatgen.io.ase as g

# Connect to database
db = connect('c2db.db')

properties = ['hform', 'ehull']
rows = [[] for value in range(len(properties))]
equation = [[] for value in range(len(properties))]
values = [[] for value in range(len(properties))]

In [11]:
def findMinEigenvalue(c11, c12, c13, c21, c22, c23, c31, c32, c33):
    matrix = [[c11, c12, c13],[c21, c22, c23],[c31, c32, c33]]
    matrix = np.array(matrix).reshape((3,3))
    return min(eigs(matrix, 3, sigma=0)[0].real)

In [30]:
temp = db.select(properties[0])
notUsed = ['He', 'Ne', 'Ar', 'Kr', 'Xe', 'Rn', 'At', 'Po', 'P', 'C', 'B', 'Tc', 'Mg', 'Be', 'Li', 'Na', 'K', 'Rb', 'Cs']

formulas = []
structures = []
hforms = []
ehulls = []
index = []

count = 0
total = 0
here = g.AseAtomsAdaptor()
stable = 0
for entry in temp:
    total+=1
    use = True
    for element in notUsed:
        if(element in entry['formula']):
            use = False
    if(use):
        try:
            hform = entry['hform']
            ehull = entry['ehull']
            formula = entry['formula']
            structure = here.get_structure(entry.toatoms())
            formulas.append(formula)
            structures.append(structure)
            if(ehull < 0.2 and hform < 0.2):
                stable += 1
            #gap = entry['gap']
            #dynamic = entry['dynamic_stability_level']
            index.append(count)
            hforms.append(hform)
            ehulls.append(ehull < 0.2)
            #structures.append(here.get_structure(entry.toatoms()))
            #print(entry)
            #print(entry['formula'])
            #print(here.get_structure(entry.toatoms()))
            count += 1
        except:
            pass
print(count)
print(total)
print(stable/count)

1778
3712
0.40888638920134984


In [31]:
from ase.db import connect
import pandas as pd
import os
from ase.db import connect
from pymatgen.core import Composition, Structure
from pymatgen.io.ase import AseAtomsAdaptor
from tqdm.autonotebook import tqdm
from scipy.sparse import identity
from scipy.sparse.linalg import eigs
import numpy as np
from aviary.cgcnn.utils import get_cgcnn_input
from aviary.wren.utils import count_wyks, get_aflow_label_spglib
from tqdm.notebook import tqdm

In [32]:
df = pd.DataFrame()
df['material_id'] = index
df['composition'] = formulas
df['final_structure'] = structures
df['wyckoff'] = df['final_structure'].apply(get_aflow_label_spglib)
df = df.drop(['final_structure'], axis=1)
df['stiffness'] = ehulls
df['thermo'] = hforms
df['stiffness'] = df["stiffness"].astype(int)

In [33]:
df_shuffled = df.sample(frac=1)
df_splits = np.array_split(df_shuffled, 20)
print(df_splits)

[      material_id composition                      wyckoff  stiffness  \
1216         1216        HfS2         AB2_hP3_187_a_h:Hf-S          0   
505           505      Zr2Te2         AB_hP4_164_d_c:Te-Zr          0   
1260         1260       NbSe2        AB2_hP3_164_a_d:Nb-Se          1   
1525         1525     Tl2I2N2      ABC_oP6_59_a_a_b:I-N-Tl          0   
200           200       Zn2S2          AB_hP4_164_d_d:S-Zn          1   
...           ...         ...                          ...        ...   
1418         1418   Hf2Zr2Te8  AB4C_oP6_25_a_ef_b:Hf-Te-Zr          1   
684           684        HgI2         AB2_tP3_115_a_g:Hg-I          1   
201           201      Zn2Se2         AB_hP4_164_d_d:Se-Zn          1   
1653         1653      Hf2Se6        AB3_oP8_59_a_be:Hf-Se          1   
1511         1511     Sc2I2S2      ABC_oP6_59_a_a_b:I-S-Sc          1   

        thermo  
1216 -1.375413  
505  -0.275863  
1260 -0.845874  
1525  0.857077  
200  -0.692252  
...        ...  
141

In [34]:
for i in range(0, 20):
    testDF = df_splits[i]
    trainDF = pd.DataFrame()
    for j in range(0, 20):
        if(i != j):
            trainDF = pd.concat([trainDF, df_splits[j]])
    trainDF.to_csv('C:\\Users\\Matthew\\Desktop\\aviary-github\\aviary\\train' + str(i) + '.csv', index=False)
    testDF.to_csv('C:\\Users\\Matthew\\Desktop\\aviary-github\\aviary\\test' + str(i) + '.csv', index=False)

In [112]:
bigThermo = pd.DataFrame()
tempDF = pd.read_csv('C:\\Users\\Matthew\\Desktop\\aviary-github\\aviary\\thermoStabilityTest.csv')
bigThermo = pd.concat([bigThermo, tempDF])
tempDF = pd.read_csv('C:\\Users\\Matthew\\Desktop\\aviary-github\\aviary\\thermoStabilityTraining.csv')
bigThermo = pd.concat([bigThermo, tempDF])

bigStiffness = pd.DataFrame()
tempDF = pd.read_csv('C:\\Users\\Matthew\\Desktop\\aviary-github\\aviary\\stiffnessTesting.csv')
bigStiffness = pd.concat([bigStiffness, tempDF])
tempDF = pd.read_csv('C:\\Users\\Matthew\\Desktop\\aviary-github\\aviary\\stiffnessTraining.csv')
bigStiffness = pd.concat([bigStiffness, tempDF])

combinedDF = pd.merge(bigThermo, bigStiffness, on='composition')
combinedDF = combinedDF.drop(['wyckoff_x'], axis=1)
combinedDF = combinedDF.drop(['material_id_y'], axis=1)
combinedDF = combinedDF.drop(['Unnamed: 0'], axis=1)
combinedDF = combinedDF.drop(['split_x'], axis=1)
combinedDF = combinedDF.drop(['split_y'], axis=1)
combinedDF = combinedDF.drop(['index'], axis=1)
print(combinedDF)
combinedDF.to_csv('thermoSitffness.csv')

      material_id_x composition    thermo                     wyckoff_y  \
0              ZrB6        ZrB6  0.402292          A6B_cP7_221_e_b:B-Zr   
1            SmSbTe      SmSbTe  0.000000    ABC_tP6_129_a_c_c:Sb-Sm-Te   
2            Mg5Si6      Mg5Si6  0.192845     A5B6_mC22_12_a2i_3i:Mg-Si   
3            Mg5Si6      Mg5Si6  0.192845     A5B6_mC22_12_a2i_3i:Mg-Si   
4            Mg5Si6      Mg5Si6  0.200840     A5B6_mC22_12_a2i_3i:Mg-Si   
...             ...         ...       ...                           ...   
86955       ZrGaPd2     ZrGaPd2  0.014161  AB2C_cF16_225_a_c_b:Ga-Pd-Zr   
86956       ZrGaPd2     ZrGaPd2  0.014161  AB2C_cF16_225_a_c_b:Ga-Pd-Zr   
86957       ZrGaPd2     ZrGaPd2  0.014161  AB2C_cF16_225_a_c_b:Ga-Pd-Zr   
86958       ZrGaPd2     ZrGaPd2  0.014161  AB2C_cF16_225_a_c_b:Ga-Pd-Zr   
86959       ZrGaPd2     ZrGaPd2  0.014161  AB2C_cF16_225_a_c_b:Ga-Pd-Zr   

       stiffness  
0              1  
1              1  
2              1  
3              1  
4   

In [7]:
for index in range(len(properties)):
    rows[index] = db.select(properties[index])
    for entry in rows[index]:
        equation[index].append(entry['formula'])
        values[index].append(entry[properties[index]])

In [77]:
for index in range(len(properties)):
    print(properties[index], " Entries: ", len(equation[index]))

hform  Entries:  4044
ehull  Entries:  4044
minhessianeig  Entries:  3859
c_11  Entries:  3968
c_12  Entries:  3968
c_13  Entries:  3968
c_21  Entries:  3968
c_22  Entries:  3968
c_23  Entries:  3968
c_31  Entries:  3968
c_32  Entries:  3968
c_33  Entries:  3968


In [78]:
intersectionResult = list(set(equation[0]).intersection(*equation))

In [79]:
print("Feasible Combined Datapoints:", len(intersectionResult))

Feasible Combined Datapoints: 2656


In [80]:
equationsTogether = intersectionResult
propertiesTogether = [[] for value in range(len(properties))]
for entry in equationsTogether:
    for index in range(len(propertiesTogether)):
        itemIndex = equation[index].index(entry)
        propertiesTogether[index].append(values[index][itemIndex])

In [81]:
print(equationsTogether[:50])

['Rh2Cl2S2', 'Ga2NiS4', 'Mn3C2O2', 'Rb2F2', 'Hg2GeO4', 'Bi2Ge6', 'Pb2I2S2', 'TiSe2', 'AsI2', 'NiS2', 'Zn2I2', 'Nb2I4', 'Mo2H2NO2', 'Ag2Br2N2', 'Cr4N3', 'Sn2O2', 'CuF2', 'Mn2P2S6', 'Ir2Te2', 'Hf2O2', 'Rh2Cl2Te2', 'Ag2F2', 'Ni4Ta2Te6', 'PbS', 'Pd2F2', 'Li2Mo', 'InTe2', 'Rh2I2N2', 'Pt2S2', 'Sb2Si6', 'Y2Te6', 'Ir2Te6', 'V2Br2O2', 'V3B2H2Se2', 'Ga2S2', 'TaCl2', 'Cr2Si2Te6', 'HfH2', 'Mo2I6', 'Fe2Te2', 'Pd2I2N2', 'V2Se4', 'Ti2Cl2', 'Re4S8', 'Mo2S6', 'Ti2PTe2', 'Co8Ta4Te8', 'Au2Br2S2', 'Mo2CH2O2', 'Os2Br2O2']


In [82]:
for entry in propertiesTogether:
    print(entry[:50])

[-0.6199853429177379, -0.5502026569670591, -0.40766834402871616, -2.598688843113121, -0.8217546279463238, 0.7274178381891461, -0.2733177640925503, -0.8611255371301437, 0.29062965168200733, 0.15065444085991805, -0.37209959656949554, -0.36938841226148433, -0.6962750875865699, 1.0472344725448182, -0.12195865271151465, -1.2562664856791708, -1.5147603015587723, -0.31004421929523074, 0.3468167948011631, -1.8850380293565951, -0.5941793656008105, -0.9132591720415464, -0.38248658470500746, -0.33679575825933483, -0.9095975339404951, 0.5032789089235953, -0.09982203730066115, 0.3696977352743283, -0.14734697179952594, 0.8499964726366587, -0.9374139054186719, -0.018625313347045314, -1.63785958883289, -0.21469018382956367, -0.21587778995398166, -0.8704703717203355, -0.2017051168711646, -0.04667963432814245, -0.21103842062687628, 0.3724385067021041, 0.6646085971535433, -0.10151547085895392, -0.7991230367307365, -0.592951799611272, -0.33531537310763326, -0.9250202462174009, -0.3609769472014605, 0.11781

In [121]:
#Not necessary
for propertyIndex in range(len(propertiesTogether)):
    for elementIndex in range(len(propertiesTogether[0])):
        element = propertiesTogether[propertyIndex][elementIndex]
        if(element == 'high'):
            propertiesTogether[propertyIndex][elementIndex] = 2
        if(element == 'low'):
            propertiesTogether[propertyIndex][elementIndex] = 1

In [83]:
minEigenvaluesFinal = []
for index in range(len(propertiesTogether[0])):
    valuesTmp = []
    for propertyIndex in range(3, len(properties)):
        valuesTmp.append(propertiesTogether[propertyIndex][index])
    minEigenvalueHere = findMinEigenvalue(valuesTmp[0],valuesTmp[1],valuesTmp[2],valuesTmp[3],valuesTmp[4],valuesTmp[5],valuesTmp[6],valuesTmp[7],valuesTmp[8])
    minEigenvaluesFinal.append(minEigenvalueHere)
    
propertiesTogether = propertiesTogether[0:len(propertiesTogether)-9]
print(len(propertiesTogether))

3


In [84]:
propertiesTogether.append(minEigenvaluesFinal)
for entry in propertiesTogether:
    print(entry[:50])

[-0.6199853429177379, -0.5502026569670591, -0.40766834402871616, -2.598688843113121, -0.8217546279463238, 0.7274178381891461, -0.2733177640925503, -0.8611255371301437, 0.29062965168200733, 0.15065444085991805, -0.37209959656949554, -0.36938841226148433, -0.6962750875865699, 1.0472344725448182, -0.12195865271151465, -1.2562664856791708, -1.5147603015587723, -0.31004421929523074, 0.3468167948011631, -1.8850380293565951, -0.5941793656008105, -0.9132591720415464, -0.38248658470500746, -0.33679575825933483, -0.9095975339404951, 0.5032789089235953, -0.09982203730066115, 0.3696977352743283, -0.14734697179952594, 0.8499964726366587, -0.9374139054186719, -0.018625313347045314, -1.63785958883289, -0.21469018382956367, -0.21587778995398166, -0.8704703717203355, -0.2017051168711646, -0.04667963432814245, -0.21103842062687628, 0.3724385067021041, 0.6646085971535433, -0.10151547085895392, -0.7991230367307365, -0.592951799611272, -0.33531537310763326, -0.9250202462174009, -0.3609769472014605, 0.11781

In [85]:
import csv
index = 0
databaseLocation = "C:\\Users\\dylan\\Desktop\\generatedData\\"
print(databaseLocation)
properties = ['hform', 'ehull', 'minhessianeig', 'minstiffeig']

with open(str(databaseLocation + 'dataCombinedRegression.csv'),'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    arrayOfEntries = ['material_id','composition']
    for propertyEntry in properties:
        arrayOfEntries.append(propertyEntry)
    csvwriter.writerow(arrayOfEntries)
    for equationTitle in equationsTogether:
        array = []
        array.append(index)
        array.append(equationsTogether[index])
        for propertyIndex in range(len(propertiesTogether)):
            array.append(propertiesTogether[propertyIndex][index])
        csvwriter.writerow(array)
        index = index + 1

C:\Users\dylan\Desktop\generatedData\


Thermodynamic:

Heat of Formation
Convex Hull

Heat > .2 eV - Low
convex + .2 < Heat < .2 eV - Medium
Heat < convex + .2 - High

Dynamic Phonons:

Minimum Eigenvalue of Hessian <= -.01 - Low
Minimum Eigenvalue of Hessian > -.01 - High

Dynamic Stiffness:

Minimum Stiffness Tensor Eigenvalue < 0 - Low
Minimum Stiffness Tensor Eigenvalue > 0 - High